# Attention is all you need 
the authors proposed a model which elminate recurrence and relying on the attention mechanism to draw global dependencies between input and output this lead to better preformance and more parallelization computing 

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# positional encoding 
we need this to compensate the absence of recurrence and to get use of the order of tokens we have to inject some information about the postion of the words so we use the sin and cos functions 

In [20]:
class positional_encoding : 
  def __init__(self) : 
    pass
  def get_angles(self , pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates
  def encoding(self , position, d_model):
    angle_rads = self.get_angles(np.arange(position)[:, np.newaxis],
                            np.arange(d_model)[np.newaxis, :],
                            d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[np.newaxis, ...]
    return tf.cast(pos_encoding, dtype=tf.float32)

# masking 
we will use two masks 
- first is padding mask to align the whole sequece length 
- second look ahead mask which help us with enforcing learning while computing in parallel this done by hashing part of the sentence which is the target for example ('I want orange ----') we hashed the word juice and it will be the target we will output 

In [21]:
class masking: 
  def __init__(self) : 
    pass
  def create_look_ahead_mask( self,size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  
  def create_padding_mask(self, seq) : 
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

# scaled dot product attetnion (self head attention) 
this the main part of the transformer which we basicly get (q, k ,v ) 
which are weighted paramters from the input and for each word we compute attention vector we compute that by multiply (q , k ) and pass the output to softmax function and multiply the output by (v) this give you good represention of each word 

A(q, k , v) = softmax(q*k)v

In [22]:
class self_head_attention(tf.keras.layers.Layer) : 
  def __init__(self ) : 
    super(self_head_attention , self).__init__()
  def call(self ,q , k , v , mask ) : 
    qk_matmul = tf.matmul(q , k , transpose_b=True) 
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = qk_matmul / tf.math.sqrt(dk)
    if mask is not None:
      scaled_attention_logits += (mask * -1e9)
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)
    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

# multi head attention 
It,s basicly repeating the self head attention to get different representions for the data 
every head is self head attention and you compute the heads in parallel and you concat them and multiply the ouput with W 


In [23]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model , num_heads): 
    super(MultiHeadAttention , self ).__init__()
    self.d_model = d_model 
    self.num_heads = num_heads 
    self.depth = d_model // self.num_heads

    # the layers to initiaize the q k v vectors 
    self.qw = tf.keras.layers.Dense(d_model)
    self.kw = tf.keras.layers.Dense(d_model) 
    self.vw = tf.keras.layers.Dense(d_model)

    self.head_attention = self_head_attention()
    self.concat_weights = tf.keras.layers.Dense(d_model)
  def split_heads(self , x , batch_size ) : 
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])
  def call(self , v ,q ,k , mask ) : 
    batch_size = tf.shape(q)[0]
    q = self.qw(q)   # (batch_size, seq_len, d_model)
    k = self.kw(k)   # (batch_size, seq_len, d_model)
    v = self.vw(v)   # (batch_size, seq_len, d_model)

    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

    scaled_attention, attention_weights = self.head_attention(
        q, k, v, mask)
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)
    output = self.concat_weights(concat_attention)
    return output, attention_weights

# feed forword 
it just basic block to pass the data throw nural networks 

In [25]:
class feed_forword_block(tf.keras.layers.Layer) :  
  def __init__(self, d_model , dff) : 
    super(feed_forword_block , self).__init__()
    self.dense_1 =  tf.keras.layers.Dense(dff, activation='relu') # (batch_size, seq_len, dff)
    self.dense_2 =  tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  def call(self , X ) : 
    X = self.dense_1(X)
    return self.dense_2(X)

# Encoder layer
the encoder layer contain 
- multi head attention 
- dropout layers 
- feed forword 
- normalization layer 

the data passed throw the multi head attention after that to the feed forword network and between there are dropout and normalization layer which also add skip connections to drive more information and keep track of the context of the word 



In [26]:
class EncoderLayer(tf.keras.layers.Layer): 
  def __init__(self, d_model, num_heads, dff, rate=0.1)  : 
    super(EncoderLayer , self).__init__()
    self.multi_head = MultiHeadAttention(d_model ,num_heads) 
    self.feed_forword = feed_forword_block(d_model , dff) 

    self.layernorm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout_1 = tf.keras.layers.Dropout(rate)
    self.dropout_2 = tf.keras.layers.Dropout(rate)

  def call(self , x , training, mask ) : 
    atte_out , _ = self.multi_head(x,x,x ,mask)
    atte_out = self.dropout_1(atte_out , training = training)
    atte_out = self.layernorm_1(  atte_out +x )
    feed_out = self.feed_forword(atte_out)
    feed_out = self.dropout_2(feed_out , training = training )
    return self.layernorm_2(x + feed_out)

# Encoder 
the encoder contain 
- embedding layer 
- positional encoding 
- num times of the encoder layer block 


In [50]:
class Encoder(tf.keras.layers.Layer): 
  def __init__(self , num_layers  , d_model , num_heads , dff ,input_vocab_size, maximum_position_encoding , rate = .1): 
      super(Encoder , self ).__init__()
      self.d_model = d_model
      self.maximum_position_encoding = maximum_position_encoding
      self.num_layers = num_layers
      self.pos = positional_encoding()
      self.embed = tf.keras.layers.Embedding(input_vocab_size , d_model ) 
      self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate)
                       for _ in range(num_layers)]

      self.dropout = tf.keras.layers.Dropout(rate)
  def call(self , x  , training ,mask) : 
    pos_encoding =self.pos.encoding(self.maximum_position_encoding, self.d_model)
    seq_len = tf.shape(x)[1]
    x = self.embed(x) # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)

    return x  # (batch_size, input_seq_len, d_model)



# Decoder 
the decoder contain 
- two multi head attention layers 
- feed forword network 
- dropout 
- normalization layers 

we pass the word throw the first multi head attention and after that we pass throw the second multi head attention the Q from the first multi head and (k , v) from the encoder output you can think about it as passing the qustion from the first multi head and getting the answers from the encoder after that we pass the ouput throw the feed forword network 

In [27]:
class DecoderLayer(tf.keras.layers.Layer) : 
  def __init__(self , d_model , num_heads , dff  ,rate) : 
    super(DecoderLayer , self).__init__() 
    self.multi_head_1 = MultiHeadAttention(d_model , num_heads)
    self.multi_head_2 = MultiHeadAttention(d_model , num_heads)
    self.feed_forword = feed_forword_block(d_model , dff)
    self.layernorm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.dropout_1 = tf.keras.layers.Dropout(rate)
    self.dropout_2 = tf.keras.layers.Dropout(rate)
    self.dropout_3 = tf.keras.layers.Dropout(rate)
  def call(self , x , encoder_output , training , look_ahead_mask , padding_mask) : 
    X_1 , attn_weights_block1= self.multi_head_1(x ,x ,x , look_ahead_mask)
    X_1 = self.dropout_1(X_1 , training = training) 
    X_1 = self.layernorm_1(x + X_1)

    X_2 ,attn_weights_block2 = self.multi_head_2(q = X_1 ,k =  encoder_output , v = encoder_output , mask =padding_mask) 
    X_2 = self.dropout_2(X_2 , training = training )
    X_2 = self.layernorm_2(x+ X_2) 
    X_3 = self.feed_forword(X_2)
    X_3 = self.dropout_3(X_3 , training = training)
    X_3 =  self.layernorm_3(x+X_3)
    return X_3, attn_weights_block1, attn_weights_block2

In [54]:
class Decoder(tf.keras.layers.Layer) : 
  def __init__(self , num_layers, d_model, num_heads, dff, target_vocab_size,
               maximum_position_encoding, rate=0.1) :
    super(Decoder , self ).__init__()
    self.d_model = d_model 
    self.maximum_position_encoding = maximum_position_encoding
    self.num_layers = num_layers
    self.embed = tf.keras.layers.Embedding(target_vocab_size  ,d_model)
    self.pos= positional_encoding()
    self.dec_layers = [DecoderLayer(d_model , num_heads , dff ,rate) for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self , x ,enc_output, training , look_ahead_mask, padding_mask ) :
    pos_encoding = self.pos.encoding(self.maximum_position_encoding, self.d_model)
    seq_len = tf.shape(x)[1]
    attention_weights = {}
    x= self.embed(x) 
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                             look_ahead_mask, padding_mask)

      attention_weights[f'decoder_layer{i+1}_block1'] = block1
      attention_weights[f'decoder_layer{i+1}_block2'] = block2

    # x.shape == (batch_size, target_seq_len, d_model)
    return x, attention_weights 


# Transformer 
this the whole model which we gather the encoder and decoder 
first we create the masks which we will need 
next we pass the data throw the model 

In [55]:
class Transformer(tf.keras.Model) : 
  def __init__(self, n_layers , d_model  ,num_heads , dff , input_vocab_size , target_vocab_size , pe_input, pe_target, rate=0.1):
    super().__init__()
    self.encoder = Encoder(n_layers , d_model  ,num_heads , dff , input_vocab_size  , rate )
    self.decoder = Decoder(n_layers , d_model  ,num_heads , dff , target_vocab_size  , rate)
    self.final_layer = tf.keras.layers.Dense(target_vocab_size) 
    self.masking = masking()
  def create_masks(self, inp , targ) : 
    # Encoder padding mask
    enc_padding_mask = self.masking.create_padding_mask(inp)

    # Used in the 2nd attention block in the decoder.
    # This padding mask is used to mask the encoder outputs.
    dec_padding_mask = self.masking.create_padding_mask(inp)

    # Used in the 1st attention block in the decoder.
    # It is used to pad and mask future tokens in the input received by
    # the decoder.
    look_ahead_mask = self.masking.create_look_ahead_mask(tf.shape(targ)[1])
    dec_target_padding_mask = self.masking.create_padding_mask(targ)
    look_ahead_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return enc_padding_mask, look_ahead_mask, dec_padding_mask 
  def call(self , inputs ,  training  ) : 
    inp, targ = inputs
    enc_padding_mask, look_ahead_mask, dec_padding_mask = self.create_masks(inp, targ)
    enc_output = self.encoder(inp , training = training , mask = enc_padding_mask)
    dec_output , attention_weights =  self.decoder(targ , enc_output  , training = training , padding_mask = dec_padding_mask , look_ahead_mask = look_ahead_mask)
    out = self.final_layer(dec_output)
    return out , attention_weights